In [33]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pygeohash
import folium

In [2]:
crime_map_df = gpd.read_file('open_data/crime_map_data.geojson')
crime_map_df = crime_map_df.to_crs("epsg:4326")
crime_map_df.head()

,OBJECTID,CASE_NO,DATE,DESCRIPTION,LOCATION,CITY,Latitude,Longitude,GlobalID,geometry
0,3,202200349491,"Fri, 25 Nov 2022 21:33:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.350226397496606,-79.80152666827507,f623ed61-fbc7-4391-8038-9e385d6ef3dd,POINT (-79.80153 43.35023)
1,10,202200351471,"Sun, 27 Nov 2022 23:46:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.33830850583127,-79.7853229172281,aff64172-9cc4-44b0-a78b-71d0c21177c8,POINT (-79.78532 43.33831)
2,13,202200353214,"Tue, 29 Nov 2022 15:40:00 GMT",MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.3646561940604,-79.8220414963383,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,POINT (-79.82204 43.36466)
3,21,202200357807,"Sun, 04 Dec 2022 17:44:48 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.34732873413244,-79.79743701462455,48573cd9-b167-4c66-8bc7-7d6e32f546ee,POINT (-79.79744 43.34733)
4,24,202200361004,"Thu, 08 Dec 2022 00:04:00 GMT",MVC - PI,GUELPH LI,BURLINGTON,43.352177632465434,-79.80386318857246,6c97ac07-1a63-41b5-95f8-725dea1811f4,POINT (-79.80386 43.35218)


In [124]:
road_segments_df = gpd.read_file('open_data/burlington_road_segments.geojson')
road_segments_df = road_segments_df.to_crs("epsg:4326")
road_segments_df.head()

,OBJECTID,STREET_NAME,STREET_TYPE,ALIAS_ST_N,L_LADD,L_HADD,R_LADD,R_HADD,STREET_CLASS,DEEMED_WIDTH,...,LEFT_COMMUNITY,RIGHT_COMMUNITY,AVLZONE_RPM,ROADDIRECTION,LAST_EDITED_DATE,WC_ZONE,ACTUAL_WIDTH,AREA40KM,SHAPELEN,geometry
0,935288,GLENWOOD AVE.,MINOR,None,800,914,775,905,U,20.0,...,BURLINGTON,BURLINGTON,South West,None,2023-02-08T20:51:59,1.0,20.0,N,623.313576,"LINESTRING (-79.83936 43.30586, -79.83976 43.3..."
1,935289,FAIRVIEW ST.,MAJOR,None,0,0,0,0,U,36.0,...,BURLINGTON,BURLINGTON,South East,None,2023-02-08T20:52:22,15.0,36.0,N,121.952115,"LINESTRING (-79.79247 43.35106, -79.79186 43.3..."
2,935290,SHARALIN CRT.,MINOR,None,569,589,570,586,U,20.0,...,BURLINGTON,BURLINGTON,South West,None,2023-02-08T20:51:49,3.0,20.0,N,67.724165,"LINESTRING (-79.82544 43.31448, -79.82485 43.3..."
3,935291,WALKER'S LINE,MAJOR,None,476,534,495,515,U,35.0,...,BURLINGTON,BURLINGTON,South East,None,2023-02-08T20:51:59,9999.0,0.0,N,353.787548,"LINESTRING (-79.76953 43.35284, -79.77167 43.3..."
4,935292,UPLAND DR.,MINOR,None,0,0,0,0,U,20.0,...,BURLINGTON,BURLINGTON,North East,None,2023-02-08T20:52:10,18.0,20.0,N,47.203260,"LINESTRING (-79.82332 43.36800, -79.82342 43.3..."


In [123]:
road_segments_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   OBJECTID          1000 non-null   int64   
 1   STREET_NAME       988 non-null    object  
 2   STREET_TYPE       1000 non-null   object  
 3   ALIAS_ST_N        60 non-null     object  
 4   L_LADD            1000 non-null   object  
 5   L_HADD            1000 non-null   object  
 6   R_LADD            1000 non-null   object  
 7   R_HADD            1000 non-null   object  
 8   STREET_CLASS      999 non-null    object  
 9   DEEMED_WIDTH      1000 non-null   float64 
 10  W_CONTROL         954 non-null    object  
 11  OWNER             1000 non-null   object  
 12  LANES             1000 non-null   int64   
 13  SPEEDLIMIT        942 non-null    float64 
 14  LEFT_COMMUNITY    1000 non-null   object  
 15  RIGHT_COMMUNITY   1000 non-null   object  
 16  AVLZONE_RPM      

In [9]:
road_segments_df.columns

Index(['OBJECTID', 'STREET_NAME', 'STREET_TYPE', 'ALIAS_ST_N', 'L_LADD',
       'L_HADD', 'R_LADD', 'R_HADD', 'STREET_CLASS', 'DEEMED_WIDTH',
       'W_CONTROL', 'OWNER', 'LANES', 'SPEEDLIMIT', 'LEFT_COMMUNITY',
       'RIGHT_COMMUNITY', 'AVLZONE_RPM', 'ROADDIRECTION', 'LAST_EDITED_DATE',
       'WC_ZONE', 'ACTUAL_WIDTH', 'AREA40KM', 'SHAPE.LEN', 'geometry'],
      dtype='object')

In [125]:
intersections_df = gpd.read_file('open_data/burlington_intersections.geojson')
intersections_df = intersections_df.to_crs("epsg:4326")
intersections_df.head()

,OBJECTID,STREET1,STREET2,STREET1_DIR,STREET2_DIR,GLOBALID,geometry
0,469884,GLENCREST RD,ROSEDALE CRES,North-South,East-West,{82F4F783-773B-4152-BB97-383DC537905A},POINT (-79.78455 43.34352)
1,469885,WOODWARD AVE,OXFORD RD,East-West,North-South,{FD42AE0E-B476-49C3-8382-AF4C2045F9F0},POINT (-79.79040 43.34314)
2,470078,FASSEL AVE,PHYLLIS ST,East-West,North-South,{90E41D6B-22B9-484B-9BCF-948505885848},POINT (-79.80764 43.34322)
3,470079,GLENCREST RD,ARLINGTON BLVD,East-West,North-South,{4F9ADBC7-B942-464A-9C57-F98ACF3B9954},POINT (-79.78529 43.34296)
4,470080,NEW ST,DYNES RD,East-West,North-South,{390FDEDB-E9FB-470B-86C8-CBCFD4FD70B8},POINT (-79.78038 43.34287)


## Geohash Conversion
We'll be using geohash level 8 as it's the geohash level at 20 metres. This should be small enough for any inaccuracies in the road segments. For segments, we'll grab the geohash of each end of the line string. 

In [128]:
intersections_df['geohash'] = intersections_df['geometry'].apply(lambda x: pygeohash.encode(x.x, x.y, precision=8)) 
crime_map_df['geohash'] = crime_map_df['geometry'].apply(lambda x: pygeohash.encode(x.x, x.y, precision=8)) 

In [129]:
def segment_geohash(line, which = 0):
    assert which == 0 or which == -1
    pos = line.coords[which]
    return pygeohash.encode(pos[0], pos[1], precision=8)
road_segments_df['geohash_1'] = road_segments_df['geometry'].apply(lambda x: segment_geohash(x, 0))
road_segments_df['geohash_2'] = road_segments_df['geometry'].apply(lambda x: segment_geohash(x, -1))
del segment_geohash

## Getting Intersection Info
Now only keep the needed info for the road segments 

In [131]:
segment_info_df = road_segments_df.query("OWNER != 'MTO'")[['STREET_NAME', 'STREET_TYPE', 'STREET_CLASS', 'ACTUAL_WIDTH', 'SPEEDLIMIT', 'LANES', 'OWNER', 'geohash_1', 'geohash_2']]
segment_info_df = pd.concat(
    [segment_info_df.rename({'geohash_1': 'geohash'}, axis=1).drop('geohash_2', axis=1),
    segment_info_df.rename({'geohash_2': 'geohash'}, axis=1).drop('geohash_1', axis=1)],
    ignore_index=True
)
segment_info_df = segment_info_df.groupby('geohash')

def set_append(items):   
    items = list(set(items))
    items.sort()
    print(items)
    return ' & '.join(items)
segment_info_df = segment_info_df.agg(
#     street_names=pd.NamedAgg(column='STREET_NAME', aggfunc=lambda x: list(set(x))),
    street_types=pd.NamedAgg(column='STREET_TYPE', aggfunc=lambda x: list(set(x))),
    owners=pd.NamedAgg(column='OWNER', aggfunc=lambda x: list(set(x))),
    max_width=pd.NamedAgg(column='ACTUAL_WIDTH', aggfunc='max'),
    max_speed_limit=pd.NamedAgg(column='SPEEDLIMIT', aggfunc='max'),
    min_speed_limit=pd.NamedAgg(column='SPEEDLIMIT', aggfunc='min'),
    max_lanes=pd.NamedAgg(column='LANES', aggfunc='max'),
    min_lanes=pd.NamedAgg(column='LANES', aggfunc='min')
)
del set_append
segment_info_df = segment_info_df.reset_index()
segment_info_df.head(n=5)

,geohash,street_types,owners,max_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes
0,hcyc3ect,[MINOR],[COB],0.0,50.0,50.0,2.0,2.0
1,hcyc3m9y,[MINOR],[COH],9999.0,NaN,NaN,2.0,2.0
2,hcyc3mgh,[MINOR],"[COB, COH]",9999.0,50.0,50.0,2.0,2.0
3,hcyc3mrv,[MINOR],[COB],0.0,50.0,50.0,2.0,2.0
4,hcyc3mun,[MINOR],[COB],0.0,50.0,50.0,2.0,2.0


Next we match the segment information to the intersections

In [132]:
intersections_df

,OBJECTID,STREET1,STREET2,STREET1_DIR,STREET2_DIR,GLOBALID,geometry,geohash
0,469884,GLENCREST RD,ROSEDALE CRES,North-South,East-West,{82F4F783-773B-4152-BB97-383DC537905A},POINT (-79.78455 43.34352),hcyf479v
1,469885,WOODWARD AVE,OXFORD RD,East-West,North-South,{FD42AE0E-B476-49C3-8382-AF4C2045F9F0},POINT (-79.79040 43.34314),hcyf469e
2,470078,FASSEL AVE,PHYLLIS ST,East-West,North-South,{90E41D6B-22B9-484B-9BCF-948505885848},POINT (-79.80764 43.34322),hcycfr3w
3,470079,GLENCREST RD,ARLINGTON BLVD,East-West,North-South,{4F9ADBC7-B942-464A-9C57-F98ACF3B9954},POINT (-79.78529 43.34296),hcyf4792
4,470080,NEW ST,DYNES RD,East-West,North-South,{390FDEDB-E9FB-470B-86C8-CBCFD4FD70B8},POINT (-79.78038 43.34287),hcyf4k3m
...,...,...,...,...,...,...,...,...
2373,472449,RAVEN AVE,EAGLE DR,East-West,North-South,{33DCE4C9-E9D9-4382-970F-294AAE9D5F96},POINT (-79.83593 43.31142),hcyccd63
2374,472450,GO STATION ACCESS,WATERDOWN RD,East-West,North-South,{2225EDF3-5F76-49D9-97E2-FEA278109DCA},POINT (-79.85886 43.31140),hcyc9w4k
2375,472451,TEAL DR,CRANSTON CRT,East-West,North-South,{35D40AD8-F60D-4206-ADE2-1459D21AB74A},POINT (-79.83760 43.30918),hcyccd0b
2376,472452,NORTH SHORE BLVD E,FOREST GLEN AVE,East-West,North-South,{B0E94127-EE12-4CBF-8C86-F155F06EF6A3},POINT (-79.83805 43.30635),hcycc3yt


In [142]:
intersections_info_df = pd.merge(intersections_df[['geohash', 'STREET1', 'STREET2']], segment_info_df, how='left', on=['geohash'])
intersections_info_df.columns = map(str.lower, intersections_info_df.columns)
intersections_info_df.head()

,geohash,street1,street2,street_types,owners,max_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,position
0,hcyf479v,GLENCREST RD,ROSEDALE CRES,[MINOR],[COB],20.0,40.0,40.0,2.0,2.0,"(-79.78451728820801, 43.3436393737793)"
1,hcyf469e,WOODWARD AVE,OXFORD RD,[MINOR],[COB],20.0,50.0,40.0,2.0,2.0,"(-79.79035377502441, 43.34329605102539)"
2,hcycfr3w,FASSEL AVE,PHYLLIS ST,[MINOR],[COB],20.0,50.0,50.0,2.0,2.0,"(-79.80769157409668, 43.34329605102539)"
3,hcyf4792,GLENCREST RD,ARLINGTON BLVD,[MINOR],[COB],20.0,40.0,40.0,2.0,2.0,"(-79.78537559509277, 43.342952728271484)"
4,hcyf4k3m,NEW ST,DYNES RD,"[MAJOR, MINOR]",[COB],20.0,60.0,40.0,4.0,2.0,"(-79.78039741516113, 43.342952728271484)"


## Export

In [150]:
crime_map_df.columns = map(str.lower, crime_map_df.columns)
pd.DataFrame(crime_map_df.drop(columns='geometry')).to_json('crash_data.json', orient='records')

In [152]:
intersections_info_df.to_json('intersection_info.json', orient='records')